# <div align="center">8.2 Challenges in Neural Network Optimization</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)

Optimization in general is an extremely difficult task. Traditionally, machine
learning has avoided the difficulty of general optimization by carefully designing
the objective function and constraints to ensure that the optimization problem is
convex. When training neural networks, we must confront the general non-convex
case. Even convex optimization is not without its complications. In this section,
we summarize several of the most prominent challenges involved in optimization
for training deep models

# <div align="center">8.2.1 ill-Conditioning</div>
---------------------------------------------------------------------

Some challenges arise even when optimizing convex functions. Of these, the most
prominent is ill-conditioning of the Hessian matrix H. This is a very general
problem in most numerical optimization, convex or otherwise. 

The ill-conditioning problem is generally believed to be present in neural
network training problems. ill-conditioning can manifest by causing SGD to get
“stuck” in the sense that even very small steps increase the cost function.

Recall that a second-order Taylor series expansion of the
cost function predicts that a gradient descent step of $−\epsilon g$ will add

<img src='asset/8_2/1.png'>

to the cost. Ill-conditioning of the gradient becomes a problem when $1/2*\epsilon H g - \epsilon g^{T} g$
exceeds $\epsilon g^{T}g$. To determine whether ill-conditioning is detrimental to a neural
network training task, one can monitor the squared gradient norm $g^{T}g$ and  the $g^{T}Hg$ term. In many cases, the gradient norm does not shrink significantly
throughout learning, but the $g^{T}Hg$ term grows by more than an order of magnitude.
The result is that learning becomes very slow despite the presence of a strong
gradient because the learning rate must be shrunk to compensate for even stronger
curvature. Figure 8.1 shows an example of the gradient increasing significantly
during the successful training of a neural network

<img src='asset/8_2/2.png'>

Figure 8.1: Gradient descent often does not arrive at a critical point of any kind. In this
example, the gradient norm increases throughout training of a convolutional network used
for object detection. (Left)A scatterplot showing how the norms of individual gradient
evaluations are distributed over time. To improve legibility, only one gradient norm
is plotted per epoch. The running average of all gradient norms is plotted as a solid
curve. The gradient norm clearly increases over time, rather than decreasing as we would
expect if the training process converged to a critical point. (Right)Despite the increasing
gradient, the training process is reasonably successful. The validation set classification
error decreases to a low level.

Though ill-conditioning is present in other settings besides neural network
training, some of the techniques used to combat it in other contexts are less
applicable to neural networks. For example, Newton’s method is an excellent tool
for minimizing convex functions with poorly conditioned Hessian matrices, but in
the subsequent sections we will argue that Newton’s method requires significant
modification before it can be applied to neural networks

# <div align="center">8.2.2 Local Minima</div>
---------------------------------------------------------------------

One of the most prominent features of a convex optimization problem is that it
can be reduced to the problem of finding a local minimum. Any local minimum is guaranteed to be a global minimum. Some convex functions have a flat region at
the bottom rather than a single global minimum point, but any point within such
a flat region is an acceptable solution. When optimizing a convex function, we
know that we have reached a good solution if we find a critical point of any kind.

***With non-convex functions, such as neural nets***, it is possible to have many
local minima. Indeed, nearly any deep model is essentially guaranteed to have
an extremely large number of local minima. However, as we will see, this is not
necessarily a major problem.

Neural networks and any models with multiple equivalently parametrized latent
variables all have multiple local minima because of the model ***identifiability
problem***. A model is said to be identifiable if a sufficiently large training set can
rule out all but one setting of the model’s parameters. Models with latent variables
are often not identifiable because we can obtain equivalent models by exchanging
latent variables with each other. For example, we could take a neural network and
modify layer 1 by swapping the incoming weight vector for unit i with the incoming
weight vector for unit j, then doing the same for the outgoing weight vectors. If we
have m layers with n units each, then there are n!m ways of arranging the hidden
units. This kind of non-identifiability is known as ***weight space symmetry***.

In addition to weight space symmetry, many kinds of neural networks have
additional causes of non-identifiability. For example, in any rectified linear or
maxout network, we can scale all of the incoming weights and biases of a unit by
α if we also scale all of its outgoing weights by 1/α. This means that—if the cost
function does not include terms such as weight decay that depend directly on the
weights rather than the models’ outputs—every local minimum of a rectified linear
or maxout network lies on an (m × n)-dimensional hyperbola of equivalent local
minima.

These model identifiability issues mean that there can be an extremely large
or even uncountably infinite amount of local minima in a neural network cost
function. However, all of these local minima arising from non-identifiability are
equivalent to each other in cost function value. As a result, these local minima are
not a problematic form of non-convexity.

Local minima can be problematic if they have high cost in comparison to the
global minimum. One can construct small neural networks, even without hidden
units, that have local minima with higher cost than the global minimum (Sontag
and Sussman, 1989; Brady et al., 1989; Gori and Tesi, 1992). If local minima
with high cost are common, this could pose a serious problem for gradient-based
optimization algorithms.

It remains an open question whether there are many local minima of high cost for networks of practical interest and whether optimization algorithms encounter
them. For many years, most practitioners believed that local minima were a
common problem plaguing neural network optimization. Today, that does not
appear to be the case. The problem remains an active area of research, but experts
now suspect that, for sufficiently large neural networks, most local minima have a
low cost function value, and that it is not important to find a true global minimum
rather than to find a point in parameter space that has low but not minimal cost.

Many practitioners attribute nearly all difficulty with neural network optimization to local minima. We encourage practitioners to carefully test for specific
problems. A test that can rule out local minima as the problem is to plot the
norm of the gradient over time. If the norm of the gradient does not shrink to
insignificant size, the problem is neither local minima nor any other kind of critical
point. This kind of negative test can rule out local minima. In high dimensional
spaces, it can be very difficult to positively establish that local minima are the
problem. Many structures other than local minima also have small gradients.

# <div align="center">8.2.3 Plateaus, Saddle Points and Other Flat Regions</div>
---------------------------------------------------------------------

For many high-dimensional non-convex functions, local minima (and maxima)
are in fact rare compared to another kind of point with zero gradient: a saddle
point. Some points around a saddle point have greater cost than the saddle point,
while others have a lower cost. At a saddle point, the Hessian matrix has both
positive and negative eigenvalues. Points lying along eigenvectors associated with
positive eigenvalues have greater cost than the saddle point, while points lying
along negative eigenvalues have lower value. We can think of a saddle point as
being a local minimum along one cross-section of the cost function and a local
maximum along another cross-section.

Many classes of random functions exhibit the following behavior: in lowdimensional spaces, local minima are common. In higher dimensional spaces, local
minima are rare and saddle points are more common. For a function f : Rn → R of
this type, the expected ratio of the number of saddle points to local minima grows
exponentially with n. To understand the intuition behind this behavior, observe
that the Hessian matrix at a local minimum has only positive eigenvalues. The
Hessian matrix at a saddle point has a mixture of positive and negative eigenvalues.
Imagine that the sign of each eigenvalue is generated by flipping a coin. In a single
dimension, it is easy to obtain a local minimum by tossing a coin and getting heads
once. In n-dimensional space, it is exponentially unlikely that all n coin tosses will be heads.

An amazing property of many random functions is that the eigenvalues of the
Hessian become more likely to be positive as we reach regions of lower cost. In
our coin tossing analogy, this means we are more likely to have our coin come up
heads n times if we are at a critical point with low cost. This means that local
minima are much more likely to have low cost than high cost. Critical points with
high cost are far more likely to be saddle points. Critical points with extremely
high cost are more likely to be local maxima.

This happens for many classes of random functions. Does it happen for neural
networks? Baldi and Hornik (1989) showed theoretically that shallow autoencoders
(feedforward networks trained to copy their input to their output, described in
chapter 14) with no nonlinearities have global minima and saddle points but no
local minima with higher cost than the global minimum. They observed without
proof that these results extend to deeper networks without nonlinearities. The
output of such networks is a linear function of their input, but they are useful
to study as a model of nonlinear neural networks because their loss function is
a non-convex function of their parameters. Such networks are essentially just
multiple matrices composed together. Saxe et al. (2013) provided exact solutions
to the complete learning dynamics in such networks and showed that learning in
these models captures many of the qualitative features observed in the training of
deep models with nonlinear activation functions. Dauphin et al. (2014) showed
experimentally that real neural networks also have loss functions that contain very
many high-cost saddle points. Choromanska et al. (2014) provided additional
theoretical arguments, showing that another class of high-dimensional random
functions related to neural networks does so as well.

What are the implications of the proliferation of saddle points for training algorithms? For first-order optimization algorithms that use only gradient information,
the situation is unclear. The gradient can often become very small near a saddle
point. On the other hand, gradient descent empirically seems to be able to escape
saddle points in many cases. Goodfellow et al. (2015) provided visualizations of
several learning trajectories of state-of-the-art neural networks, with an example
given in figure 8.2. These visualizations show a flattening of the cost function near
a prominent saddle point where the weights are all zero, but they also show the
gradient descent trajectory rapidly escaping this region. Goodfellow et al. (2015)
also argue that continuous-time gradient descent may be shown analytically to be
repelled from, rather than attracted to, a nearby saddle point, but the situation
may be different for more realistic uses of gradient descent.

For Newton’s method, it is clear that saddle points constitute a problem. 

<img src='asset/8_2/3.png'>

Figure 8.2: A visualization of the cost function of a neural network. Image adapted
with permission from Goodfellow et al. (2015). These visualizations appear similar for
feedforward neural networks, convolutional networks, and recurrent networks applied
to real object recognition and natural language processing tasks. Surprisingly, these
visualizations usually do not show many conspicuous obstacles. Prior to the success of
stochastic gradient descent for training very large models beginning in roughly 2012,
neural net cost function surfaces were generally believed to have much more non-convex
structure than is revealed by these projections. The primary obstacle revealed by this
projection is a saddle point of high cost near where the parameters are initialized, but, as
indicated by the blue path, the SGD training trajectory escapes this saddle point readily.
Most of training time is spent traversing the relatively flat valley of the cost function,
which may be due to high noise in the gradient, poor conditioning of the Hessian matrix
in this region, or simply the need to circumnavigate the tall “mountain” visible in the
figure via an indirect arcing path

Gradient descent is designed to move “downhill” and is not explicitly designed
to seek a critical point. Newton’s method, however, is designed to solve for a
point where the gradient is zero. Without appropriate modification, it can jump
to a saddle point. The proliferation of saddle points in high dimensional spaces
presumably explains why second-order methods have not succeeded in replacing
gradient descent for neural network training. Dauphin et al. (2014) introduced a
saddle-free Newton method for second-order optimization and showed that it
improves significantly over the traditional version. Second-order methods remain
difficult to scale to large neural networks, but this saddle-free approach holds
promise if it could be scaled.

There are other kinds of points with zero gradient besides minima and saddle
points. There are also maxima, which are much like saddle points from the
perspective of optimization—many algorithms are not attracted to them, but
unmodified Newton’s method is. Maxima of many classes of random functions
become exponentially rare in high dimensional space, just like minima do.

There may also be wide, flat regions of constant value. In these locations, the
gradient and also the Hessian are all zero. Such degenerate locations pose major
problems for all numerical optimization algorithms. In a convex problem, a wide,
flat region must consist entirely of global minima, but in a general optimization
problem, such a region could correspond to a high value of the objective function

# <div align="center">8.2.4 Cliffs and Exploding Gradients</div>
---------------------------------------------------------------------

Neural networks with many layers often have extremely steep regions resembling
cliffs, as illustrated in figure 8.3. These result from the multiplication of several
large weights together. On the face of an extremely steep cliff structure, the
gradient update step can move the parameters extremely far, usually jumping off
of the cliff structure altogether.

<img src='asset/8_2/4.png'>

Figure 8.3: The objective function for highly nonlinear deep neural networks or for
recurrent neural networks often contains sharp nonlinearities in parameter space resulting
from the multiplication of several parameters. These nonlinearities give rise to very
high derivatives in some places. When the parameters get close to such a cliff region, a
gradient descent update can catapult the parameters very far, possibly losing most of the
optimization work that had been done. Figure adapted with permission from Pascanu
et al. (2013)

# <div align="center">8.2.5 Long-Term Dependencies</div>
---------------------------------------------------------------------

Another difficulty that neural network optimization algorithms must overcome
arises when the computational graph becomes extremely deep. Feedforward
networks with many layers have such deep computational graphs.

For example, suppose that a computational graph contains a path that consists
of repeatedly multiplying by a matrix W. After t steps, this is equivalent to multiplying by $W^t$ . Suppose that W has an eigendecomposition W = V diag(λ)V −1.
In this simple case, it is straightforward to see that

<img src='asset/8_2/5.png'>

Any eigenvalues $λ_{i}$ that are not near an absolute value of 1 will either explode if they
are greater than 1 in magnitude or vanish if they are less than 1 in magnitude. The
***vanishing and exploding gradient problem*** refers to the fact that gradients
through such a graph are also scaled according to diag(λ)t. Vanishing gradients
make it difficult to know which direction the parameters should move to improve
the cost function, while exploding gradients can make learning unstable. The cliff
structures described earlier that motivate gradient clipping are an example of the
exploding gradient phenomenon

The repeated multiplication by W at each time step described here is very
similar to the power method algorithm used to find the largest eigenvalue of
a matrix W and the corresponding eigenvector. From this point of view it is
not surprising that $x^{T}W_{t}$ will eventually discard all components of x that are
orthogonal to the principal eigenvector of W.

Recurrent networks use the same matrix W at each time step, but feedforward
networks do not, so even very deep feedforward networks can largely avoid the
vanishing and exploding gradient problem (Sussillo, 2014).